Dependencies: camelot-py[base], opencv-python, ghostscript, pyPDF~=2.0, matplotlib

In [ ]:

import camelot 
import pandas as pd
import numpy as np
import os

In [ ]:
Directory = "E:\\Data analytics projects\\Financial Tracker\\Statements"
PDF_list = [os.path.join(Directory, x) for x in os.listdir(Directory)]
print(PDF_list)
print(len(PDF_list))

In [ ]:
##table_areas=['50, 770,500,0 ']
#camelot.plot(tables[2], kind='text').show()

file_path = str(PDF_list[5])
tables = camelot.read_pdf(file_path, pages='all', flavor='stream')
all_reports = []

# Print accuracy report and save a txt file with aggregated report contents.
for x in range(len(tables)):
    table = tables[x]
    report = table.parsing_report
    all_reports.append(f"Table {x} Report:\n{report}\n{'='*40}\n")

filename = os.path.basename(file_path)  # Extract the filename from the full path
report_file_path = f"E:/Data analytics projects/Financial Tracker/Accuracy Reports/{filename}_aggregate.txt"

# Create a single text file with information about all tables
with open(report_file_path, "w") as report_file:
    report_file.write(f"{file_path} Accuracy Report\nNumber of tables: {len(tables)}\n\n")
    report_file.write("".join(all_reports))

print(f"Aggregate accuracy report saved to: {report_file_path}")

In [ ]:
top_df = tables[1].df

# Create a DataFrame from top_df
df = pd.DataFrame(data=top_df)

# Find the index of the row that contains the header information
header_index = df[df.iloc[:, :2].eq('Date').any(axis=1)].index[0]

# Promote the row as the header
df.columns = df.iloc[header_index]

# Remove all rows before the header row
df = df.iloc[header_index + 1:, :]

# Remove all columns before the 'Date' column and all columns after the 5th column
df = df.loc[:, 'Date':].iloc[:, :5]


# Reset the index
df.reset_index(drop=True, inplace=True)

# df now contains the cleaned and restructured DataFrame
df.shape

In [ ]:
dataframes = {}

for x in range(2, len(tables)):
    loop_df = tables[x].df
    loop_df = pd.DataFrame(data=loop_df)

    # Check if the DataFrame has at least 3 rows and 5 columns
    if loop_df.shape[0] > 1 and loop_df.shape[1] > 4:
        try:
            # Find the index of the row that contains the header information
            header_index = loop_df[loop_df.iloc[:, :2].eq('Date').any(axis=1)].index[0]
            
            # Promote the row as the header
            loop_df.columns = loop_df.iloc[header_index]
            
            # Remove all rows before the header row
            loop_df = loop_df.iloc[header_index + 1:, :]
            
            # Remove all columns before the 'Date' column
            loop_df = loop_df.loc[:, 'Date':]
            
            # Save the cleaned DataFrame to the dictionary
            dataframes[f"df{x}"] = loop_df

        except IndexError:
            print(f"Error: Header row not found in df{x}. Skipping...")
            print(loop_df.head())  # Print the head of loop_df
            continue


print(dataframes)


In [ ]:
if len(dataframes) > 0:
    # Concatenate all DataFrames in the dictionary
    bottom_df = pd.concat(dataframes.values())

    # Reset the index
    bottom_df.reset_index(drop=True, inplace=True)
    
    # Concatenate df and bottom_df to create main_df
    main_df = pd.concat([df, bottom_df])
    
    # Print the expected shape before concatenation
    print(f'Expected shape = {df.shape + bottom_df.shape}')
    
    # Print the shape of the concatenated main_df
    print(f'main_df shape: {main_df.shape}')
    
    # Display the last few rows of main_df
    main_df.tail()
    
else:
    main_df = df 
    print("There's only one table in the PDF")

In [ ]:
main_df.replace('', np.nan, inplace=True)
main_df = main_df.dropna(how='all')  # Delete empty rows
main_df['is_empty'] = main_df[['Date', 'withdrawn ($)', 'deposited ($)', 'Balance ($)']].isnull().all(axis=1)
main_df['is_empty'].value_counts()
main_df = main_df.reset_index(drop=True)
main_df.head()

In [ ]:
i = 0
while i < len(main_df):
    if main_df['is_empty'].iloc[i] == True:
        # Concatenate Transactions with the previous row
        main_df['Transactions'].iloc[i-1] += " " + main_df['Transactions'].iloc[i]
        
        # Drop the current row after concatenating Transactions
        main_df = main_df.drop(main_df.index[i])
        main_df = main_df.reset_index(drop=True)
    else:
        i += 1

# Drop the 'is_empty' column as it's no longer needed
main_df = main_df.drop(columns=['is_empty'])

# Display the first few rows of the modified DataFrame
main_df.head()


In [ ]:
#Sometimes, there's a remaining date row with just year value. We have to account for that
main_df['Date'] = main_df['Date'] + ' ' + str(2023)
main_df['Date'] = pd.to_datetime(main_df['Date'], format='%b %d %Y')
try:
    main_df.loc[main_df['Date'].dt.month == 1, 'Date'] = main_df['Date'] + pd.DateOffset(years=1)
except:
    pass

main_df.head()

In [ ]:
output_csv_path = "E:\Data analytics projects\Financial Tracker\main_df.csv"
main_df.to_csv(output_csv_path, index=False, mode='a', header=not os.path.exists(output_csv_path))